In [ ]:
from aries_askar import Key, KeyAlg
from didcomm_messaging.crypto.backend.askar import AskarCryptoService, AskarSecretKey
from didcomm_messaging.crypto.backend.basic import InMemorySecretsManager
from didcomm_messaging.packaging import PackagingService
from didcomm_messaging.multiformats import multibase
from didcomm_messaging.multiformats import multicodec
from didcomm_messaging.resolver.peer import Peer2, Peer4
from didcomm_messaging.resolver import PrefixResolver
from did_peer_2 import KeySpec, generate, json
from didcomm_messaging import DIDCommMessaging
from didcomm_messaging.routing import RoutingService
from pydid.did import DID


In [ ]:
secrets = InMemorySecretsManager()
crypto = AskarCryptoService()
packaging = PackagingService()
resolver = PrefixResolver({"did:peer:2": Peer2(), "did:peer:4": Peer4()})

verkey = Key.generate(KeyAlg.ED25519)
xkey = Key.generate(KeyAlg.X25519)
did = generate(
    [
        KeySpec.verification(
            multibase.encode(
                multicodec.wrap("ed25519-pub", verkey.get_public_bytes()),
                "base58btc",
            )
        ),
        KeySpec.key_agreement(
            multibase.encode(
                multicodec.wrap("x25519-pub", xkey.get_public_bytes()), "base58btc"
            )
        ),
    ],
    [
                {
                    "type": "DIDCommMessaging",
                    "serviceEndpoint": {
                        "uri": "ws://localhost:8765",
                        "accept": ["didcomm/v2"],
                        "routingKeys": [],
                    },
                },
    ],
)
await secrets.add_secret(AskarSecretKey(verkey, f"{did}#key-1"))
await secrets.add_secret(AskarSecretKey(xkey, f"{did}#key-2"))
print(did)
message = b"hello world"
packed = await packaging.pack(crypto, resolver, secrets, message, [did], did)
print(json.dumps(json.loads(packed), indent=2))
unpacked = await packaging.unpack(crypto, resolver, secrets, packed)
print(unpacked)

In [ ]:
router = RoutingService()

dmp = DIDCommMessaging(
    crypto=crypto,
    secrets=secrets,
    resolver=resolver,
    packaging=packaging,
    routing=router,
)


In [ ]:
did

In [ ]:
import uuid
message = {
        "type": "https://didcomm.org/musig2/join",
        "id": str(uuid.uuid4()),
        "body": {},
        "from": did,
        "to": [did],
    }

In [ ]:
packy = await dmp.pack(
        message=message,
        to=did,
        frm=did,
    )
packed = packy.message

# Get the first http endpoint from the last DID in the DID chain
endpoint = packy.get_endpoint("ws")

In [ ]:
packed

In [ ]:
import asyncio
import websockets

# This handler will be called for every client connection.
async def echo(websocket):
    print("Client connected")
    try:
        async for message in websocket:
            print(f"Received: {message}")
            # Echo the received message back to the client.
            await websocket.send(f"Echo: {message}")

            unpacked =  await dmp.packaging.unpack(crypto, resolver, secrets,message)
            print(f"Unpacked Message : {unpacked}")
    except websockets.exceptions.ConnectionClosed as e:
        print("Connection closed", e)

# The main function to start the server.
async def run_websocket():
    # Start the server on localhost at port 8765.
    async with websockets.serve(echo, "localhost", 8765):
        print("WebSocket server started on ws://localhost:8765")
        # Run the server until it is manually stopped.
        await asyncio.Future()  # Run forever



In [ ]:
asyncio.create_task(run_websocket())

In [ ]:
async def send_message(endpoint, packed):
    async with websockets.connect(endpoint) as websocket:
        message = "Hello from the client!"
        print(f"Sending message: {packed}")
        await websocket.send(packed)  # Send message to the server
        
        response = await websocket.recv()  # Wait for the server's response
        print(f"Received response: {response}")

In [ ]:
asyncio.create_task(send_message(endpoint, packed))

In [ ]:
endpoint